In [138]:
import pandas as pd
import numpy as np
from metrics_f1 import calc_f1_score

In [139]:
path_train = r"./"

In [140]:
kti1 = pd.read_parquet(path_train + '/kti_izm.parquet').convert_dtypes()
kti2 = pd.read_parquet('../train2/kti_izm.parquet').convert_dtypes()
kti = pd.concat([kti1, kti2])

In [141]:
kti['wagnum'].duplicated().sum()

82561

In [142]:
kti.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90395 entries, 0 to 212366
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   wagnum               90395 non-null  Int64  
 1   operation_date_dttm  90395 non-null  Int64  
 2   mileage_all          89336 non-null  Int64  
 3   axl1_l_w_flange      90360 non-null  Float64
 4   axl1_r_w_flange      90332 non-null  Float64
 5   axl2_l_w_flange      90355 non-null  Float64
 6   axl2_r_w_flange      90340 non-null  Float64
 7   axl3_l_w_flange      90350 non-null  Float64
 8   axl3_r_w_flange      90332 non-null  Float64
 9   axl4_l_w_flange      90360 non-null  Float64
 10  axl4_r_w_flange      90344 non-null  Float64
 11  axl1_l_w_rim         89987 non-null  Float64
 12  axl1_r_w_rim         89986 non-null  Float64
 13  axl2_l_w_rim         89985 non-null  Float64
 14  axl2_r_w_rim         89933 non-null  Float64
 15  axl3_l_w_rim         89915 non-null  Flo

In [143]:
kti.isna().sum()

wagnum                    0
operation_date_dttm       0
mileage_all            1059
axl1_l_w_flange          35
axl1_r_w_flange          63
axl2_l_w_flange          40
axl2_r_w_flange          55
axl3_l_w_flange          45
axl3_r_w_flange          63
axl4_l_w_flange          35
axl4_r_w_flange          51
axl1_l_w_rim            408
axl1_r_w_rim            409
axl2_l_w_rim            410
axl2_r_w_rim            462
axl3_l_w_rim            480
axl3_r_w_rim            465
axl4_l_w_rim            354
axl4_r_w_rim            379
dtype: int64

In [144]:
kti.describe()

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
count,90395.0,90395.0,89336.0,90360.0,90332.0,90355.0,90340.0,90350.0,90332.0,90360.0,90344.0,89987.0,89986.0,89985.0,89933.0,89915.0,89930.0,90041.0,90016.0
mean,19517.78948,1671265499976768512.0,78675.160797,29.097934,29.070724,29.095071,29.157958,29.082714,29.021269,29.17571,29.22139,62.259626,62.267024,61.223539,61.242478,61.22515,61.225034,62.250035,62.246842
std,8530.396802,4517743659359157.0,51503.212358,1.689831,1.687698,1.739701,1.697841,1.678357,1.700114,1.741217,1.707543,11.64811,11.664029,12.129787,12.119589,12.114947,12.103777,11.646075,11.636527
min,0.0,1659312000000000000.0,1.0,23.4,23.0,23.1,22.7,23.4,21.8,23.3,22.9,23.6,22.8,20.7,23.3,23.2,23.6,24.1,23.5
25%,13753.0,1668124800000000000.0,29680.0,28.0,27.9,27.9,28.0,28.0,27.9,28.0,28.1,55.6,55.7,54.0,54.0,53.9,54.0,55.5,55.5
50%,19912.0,1672012800000000000.0,79215.0,29.1,29.0,29.1,29.1,29.0,29.0,29.1,29.2,64.9,64.9,63.6,63.6,63.6,63.6,64.9,64.9
75%,26123.0,1675123200000000000.0,124645.25,30.2,30.1,30.2,30.2,30.2,30.1,30.3,30.3,72.3,72.4,72.1,72.0,72.1,72.1,72.4,72.3
max,33975.0,1677542400000000000.0,504146.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0,80.7,80.4,80.3,80.1,80.2,80.0,80.7,80.0


Drop rows if we don't have any info about the wagnum(иначе какому вагону предсказывать) or date of measure kti(измерение может быть уже не актуальным или в будущем для трейна)

In [145]:
before = len(kti)
kti = kti.dropna(subset=['wagnum', 'operation_date_dttm'])
after = len(kti)
before - after

0

общий пробег заменим средним

In [146]:
kti['mileage_all'].fillna(round(kti['mileage_all'].mean()), inplace=True)

для всех остальных тоже заменим средним

In [147]:
float_columns = kti.select_dtypes(include='float').columns

for col in float_columns:
    kti[col].fillna(kti[col].mean(), inplace=True)

In [148]:
kti.isna().sum()

wagnum                 0
operation_date_dttm    0
mileage_all            0
axl1_l_w_flange        0
axl1_r_w_flange        0
axl2_l_w_flange        0
axl2_r_w_flange        0
axl3_l_w_flange        0
axl3_r_w_flange        0
axl4_l_w_flange        0
axl4_r_w_flange        0
axl1_l_w_rim           0
axl1_r_w_rim           0
axl2_l_w_rim           0
axl2_r_w_rim           0
axl3_l_w_rim           0
axl3_r_w_rim           0
axl4_l_w_rim           0
axl4_r_w_rim           0
dtype: int64

No duplicates

In [149]:
kti[kti.duplicated()]

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim


No duplicate measures same wagon in same time

In [150]:
duplicates = kti[kti.duplicated(subset=['wagnum', 'operation_date_dttm'])]
duplicates

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim


How many times measured each wagon

In [151]:
kti.groupby('wagnum')['operation_date_dttm'].count().describe()

count    7834.000000
mean       11.538805
std         7.913427
min         1.000000
25%         5.000000
50%        10.000000
75%        16.000000
max        48.000000
Name: operation_date_dttm, dtype: float64

nanosecond -> seconds

In [152]:
kti['operation_date_dttm'] = kti['operation_date_dttm'].apply(lambda x: x // 10**9)

In [153]:
grouped_by_wagnum = kti.groupby('wagnum')


In [154]:
dataframe_group = None

for key, group in grouped_by_wagnum:
    dataframe_group = group
    break

dataframe_group = dataframe_group.sort_values(by='operation_date_dttm')
dataframe_group

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
109258,0,1667174400,91119,26.8,27.9,25.5,26.2,27.3,26.2,26.9,27.5,48.0,49.4,42.6,45.6,42.5,44.4,42.9,47.2
109259,0,1667260800,91119,26.9,28.2,24.7,26.6,27.8,26.2,26.8,27.5,46.4,49.6,44.3,46.7,43.1,44.1,43.9,47.3
109260,0,1668902400,95765,27.2,28.4,25.2,26.2,27.4,26.2,27.1,27.8,46.4,47.3,43.4,45.6,42.8,42.9,41.8,43.9
109261,0,1671580800,97300,26.7,27.8,27.9,27.1,32.9,28.9,30.7,31.0,48.7,48.0,44.8,45.0,30.8,31.8,39.7,39.8
109262,0,1671753600,98318,26.7,28.2,27.8,27.2,31.8,29.1,30.7,31.5,48.3,46.7,46.3,43.5,30.2,28.2,38.8,38.1
109263,0,1672099200,100204,26.9,28.5,27.4,26.8,31.3,29.4,30.3,31.2,48.9,48.1,45.2,44.7,29.0,28.0,37.7,39.6
109264,0,1672185600,100972,26.3,28.1,27.7,26.5,31.4,28.7,29.6,30.6,48.7,46.9,43.6,42.8,28.6,29.1,38.4,39.0
109265,0,1672272000,101432,26.8,28.4,27.6,26.4,32.2,29.7,29.8,30.8,47.7,48.5,44.0,42.8,29.3,26.5,38.4,38.4
109266,0,1673568000,106028,26.8,28.4,27.8,26.9,31.8,29.5,30.1,30.9,47.9,48.7,42.6,43.8,29.6,30.1,37.1,39.2
109267,0,1675382400,108372,26.1,28.0,27.3,25.9,31.2,29.3,29.5,30.2,48.8,47.3,42.0,43.3,29.2,29.9,37.6,38.5


In [155]:
kti.to_csv('../preprocessed/kti.csv')

In [126]:
from datetime import datetime
time_of_measure = np.vectorize(lambda x: (x, datetime.utcfromtimestamp(x)))(np.unique(kti['operation_date_dttm']))
time_of_measure

(array([1659312000, 1659398400, 1659484800, 1659571200, 1659657600,
        1659744000, 1659830400, 1659916800, 1660003200, 1660089600,
        1660176000, 1660262400, 1660348800, 1660435200, 1660521600,
        1660608000, 1660694400, 1660780800, 1660867200, 1660953600,
        1661040000, 1661126400, 1661212800, 1661299200, 1661385600,
        1661472000, 1661558400, 1661644800, 1661731200, 1661817600,
        1661904000, 1661990400, 1662076800, 1662163200, 1662249600,
        1662336000, 1662422400, 1662508800, 1662595200, 1662681600,
        1662768000, 1662854400, 1662940800, 1663027200, 1663113600,
        1663200000, 1663286400, 1663372800, 1663459200, 1663545600,
        1663632000, 1663718400, 1663804800, 1663891200, 1663977600,
        1664064000, 1664150400, 1664236800, 1664323200, 1664409600,
        1664496000, 1664582400, 1664668800, 1664755200, 1664841600,
        1664928000, 1665014400, 1665100800, 1665187200, 1665273600,
        1665360000, 1665446400, 1665532800, 1665

In [29]:
train = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
train

,wagnum,month,target_month,target_day
0,33361,2023-01-01,0,0
1,33364,2023-01-01,0,0
2,33366,2023-01-01,0,0
3,33358,2023-01-01,0,0
4,33349,2023-01-01,0,0
...,...,...,...,...
203848,25045,2022-12-01,0,0
203849,27156,2022-12-01,0,0
203850,21361,2022-12-01,0,0
203851,8061,2022-12-01,0,0


In [30]:
import sys
sys.path.append('..')
from src.timeconverter import date_to_timestamp
train['month'] = train['month'].apply(date_to_timestamp)
train

,wagnum,month,target_month,target_day
0,33361,1672531200,0,0
1,33364,1672531200,0,0
2,33366,1672531200,0,0
3,33358,1672531200,0,0
4,33349,1672531200,0,0
...,...,...,...,...
203848,25045,1669852800,0,0
203849,27156,1669852800,0,0
203850,21361,1669852800,0,0
203851,8061,1669852800,0,0


In [31]:
train_time_of_measure = np.vectorize(lambda x: (x, datetime.utcfromtimestamp(x)))(np.unique(train['month']))
train_time_of_measure

(array([1659312000, 1661990400, 1664582400, 1667260800, 1669852800,
        1672531200]),
 array([datetime.datetime(2022, 8, 1, 0, 0),
        datetime.datetime(2022, 9, 1, 0, 0),
        datetime.datetime(2022, 10, 1, 0, 0),
        datetime.datetime(2022, 11, 1, 0, 0),
        datetime.datetime(2022, 12, 1, 0, 0),
        datetime.datetime(2023, 1, 1, 0, 0)], dtype=object))

In [32]:
combined_train = pd.merge(train, kti, on='wagnum', how='right')
combined_train

,wagnum,month,target_month,target_day,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,1672531200,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
1,325,1659312000,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
2,325,1661990400,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
3,325,1664582400,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
4,325,1667260800,1,1,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407797,28090,1659312000,0,0,1675123200,158013,28.7,26.6,28.0,28.7,...,31.1,28.1,59.3,61.2,64.5,65.4,70.9,71.8,72.1,73.5
407798,28090,1661990400,0,0,1675123200,158013,28.7,26.6,28.0,28.7,...,31.1,28.1,59.3,61.2,64.5,65.4,70.9,71.8,72.1,73.5
407799,28090,1664582400,0,0,1675123200,158013,28.7,26.6,28.0,28.7,...,31.1,28.1,59.3,61.2,64.5,65.4,70.9,71.8,72.1,73.5
407800,28090,1667260800,0,0,1675123200,158013,28.7,26.6,28.0,28.7,...,31.1,28.1,59.3,61.2,64.5,65.4,70.9,71.8,72.1,73.5


In [33]:
combined_train = combined_train[combined_train['operation_date_dttm'] <= combined_train['month']]
combined_train

,wagnum,month,target_month,target_day,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,...,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,1672531200,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
1,325,1659312000,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
2,325,1661990400,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
3,325,1664582400,0,0,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
4,325,1667260800,1,1,1659312000,112091,30.5,31.0,26.7,27.4,...,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391542,26252,1672531200,0,0,1672531200,2095,28.9,30.1,29.3,30.4,...,29.3,29.2,69.6,69.9,67.7,68.1,60.6,60.7,60.3,60.2
391566,28878,1672531200,0,0,1672531200,130212,27.6,26.4,26.5,27.4,...,26.8,27.3,54.8,55.4,55.1,55.9,60.4,61.9,55.3,55.7
391608,32663,1672531200,0,0,1672531200,80609,32.1,30.0,30.0,31.1,...,30.7,30.7,69.9,70.7,72.5,72.3,71.7,74.3,71.6,72.4
391644,29878,1672531200,0,0,1672531200,1092,30.6,30.1,30.7,29.9,...,30.3,30.4,58.9,58.0,57.1,57.3,61.6,59.8,56.9,57.0


In [36]:
combined_train.isna().sum()

wagnum                 0
month                  0
target_month           0
target_day             0
operation_date_dttm    0
mileage_all            0
axl1_l_w_flange        0
axl1_r_w_flange        0
axl2_l_w_flange        0
axl2_r_w_flange        0
axl3_l_w_flange        0
axl3_r_w_flange        0
axl4_l_w_flange        0
axl4_r_w_flange        0
axl1_l_w_rim           0
axl1_r_w_rim           0
axl2_l_w_rim           0
axl2_r_w_rim           0
axl3_l_w_rim           0
axl3_r_w_rim           0
axl4_l_w_rim           0
axl4_r_w_rim           0
dtype: int64

## *kti measure are in future of train!!*